In [4]:
from itertools import product
import gc
import random
import json
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import igraph as ig
import numpy as np
from igraph import *
import tqdm as tq
from tqdm import tqdm
import multiprocessing as mp
from datetime import datetime

argumenti se dobivaju od master bilježnice

In [6]:

VERSION=str(MIN_CLANAKA)+'-'+str(MAX_CLANAKA)+'-'+str(BROJ_KORISNIKA)+'-'+str(MAX_DEGREE)+"_ID@_"+str(ID)
VERSION


'6-10-200000-100_ID@_5'

In [7]:
!cd /home/dora/Desktop/recommender/datasets/master_generated && mkdir $VERSION

mkdir: cannot create directory ‘6-10-200000-100_ID@_5’: File exists


In [8]:
USERS_JSON_PATH = "/home/dora/Desktop/recommender/datasets/master_generated/"+VERSION+"/random_walk_"+VERSION+".json"

In [9]:
USERS_JSON_PATH

'/home/dora/Desktop/recommender/datasets/master_generated/6-10-200000-100_ID@_5/random_walk_6-10-200000-100_ID@_5.json'

In [10]:
try:
    edges 
except NameError: #možemo više puta pokretati bez da se učitava ispočetka
    print("ucitavanje grafa..")
    with open('/home/dora/Desktop/recommender/datasets/edges.json', 'r') as f:
        edges=json.load(f)
    with open('/home/dora/Desktop/recommender/datasets/vertices.json', 'r') as f:
        vertices=json.load(f)
    with open('/home/dora/Desktop/recommender/datasets/weights.json', 'r') as f:
        weights=json.load(f)
    
    #init graf
    G=Graph() 
    G.add_vertices(vertices)
    G.add_edges(edges)
    G.es["weight"]=weights
    #maxG=G.clusters().giant() #sa ovime se dalje radi

ucitavanje grafa..


In [11]:
G.summary()

'IGRAPH UNW- 3851441 35708851 -- \n+ attr: name (v), weight (e)'

In [12]:
maxG=G

In [13]:
np.median(maxG.degree())

10.0


___


### micanje survey i članaka stupnja većeg od 100

In [14]:
import json
with open('/home/dora/Desktop/recommender/helper_notebooks/survey_ids.json', 'r') as f:
        bad_ids=json.load(f)

In [15]:
to_delete_ids = []
for b in bad_ids:
    try:
        to_delete_ids.append(maxG.vs.find(name=b).index)
    except:
        pass


In [16]:
maxG.summary()

'IGRAPH UNW- 3851441 35708851 -- \n+ attr: name (v), weight (e)'

In [17]:
maxG.delete_vertices(to_delete_ids)

In [18]:
maxG.summary()

'IGRAPH UNW- 3734632 32438191 -- \n+ attr: name (v), weight (e)'

**maknuti cemo i vrhove stupnja veceg od 100**

In [19]:
vertices=maxG.vs

bad_nodes_ids=[]
for v in vertices:
    if v.degree() > MAX_DEGREE:
        bad_nodes_ids.append(v.index)

maxG.delete_vertices(bad_nodes_ids)

In [20]:
maxG.summary() #bez micanja 3851441 35708851

'IGRAPH UNW- 3678922 20212693 -- \n+ attr: name (v), weight (e)'

In [21]:
maxG.vcount()

3678922

___

In [22]:
maxG=maxG.clusters().giant()
maxG.summary()

'IGRAPH UNW- 3191507 20197198 -- \n+ attr: name (v), weight (e)'

___

In [43]:
assert MIN_HOPS_DIFF < MAX_HOPS_DIFF, f"Outer ring 'radius' must be greater than that of an inner ring!"

def get_articles_from_processes(res):
    global clanci_svih_korisnika, anh, aph

    clanci_svih_korisnika.extend(res[0])
    anh.extend(res[1])
    aph.extend(res[2])
    
    
    

def get_hard_neg(init_clanci, node_hops_origin, tmp_max):
    min_hops = tmp_max + MIN_HOPS_DIFF
    max_hops = tmp_max + MAX_HOPS_DIFF
    negatives = set()
    eps=0.00001
    front=init_clanci.copy()
    visited= init_clanci.copy()
    br=0
    all_hops=0
    while len(negatives) <= NUM_HARD_NEG:
            random_node = random.choice(list(front)) # rand node
            visited.add(random_node)
            last_hops=node_hops_origin[random_node] + 1
            neighbors = set(maxG.neighbors(random_node))  # svi njegovi susjedi
            unknown_neighbors = list(neighbors.difference(visited))  #dohvati one koje nismo posjetili
            front.remove(random_node) 


            if len(unknown_neighbors) == 0: #ako su već svi u listi
                continue                 #izaberi drugi random node

           
                  
            #remove_nodes=set()
               
            # for node in front:
            #     if node_hops_origin[node] < (last_hops - hop_alpha):
            #         remove_nodes.add(node)
            #        if len(remove_nodes) > len(front) * 1/3:
            #            break
                    
            #front -= remove_nodes       #speed + distance
            
            weights = []
            for n in unknown_neighbors:
                if node not in node_hops_origin:
                    node_hops_origin[node]=last_hops
                eid = maxG.get_eid(random_node, n)
                weights.append(maxG.es[eid]['weight'])
                #weights.append(node_hops_origin[n])

            max_fitness = max(weights)*1.0
            
            for f in range(0, len(weights)):
                weights[f] = (max_fitness - weights[f] + eps) 


            max_value = sum(weights)

            for f in range(0, len(weights)):
                weights[f] = weights[f]  / max_value
                if f > 0:
                    weights[f] = weights[f] + weights[f - 1]

            weights = np.around(weights, 5)
            pick = random.uniform(0, 1)

            position = np.searchsorted(weights, pick)

            random_node2 = unknown_neighbors[position]
            
            front.update(unknown_neighbors) #dodajemo samo one koji nisu jos posjeceni
            if last_hops >= min_hops and last_hops <= max_hops:
                negatives.add(random_node2)
                br+=1
                all_hops+=last_hops
                
                        
    
    return  negatives, all_hops/br
    

In [44]:
def walk():
    global pbar
    all_nodes_ids=list(range(maxG.vcount()))
    all_articles=[]
    all_neg_avg_hops=[]
    all_pos_avg_hops=[]
    while len(all_articles) < BROJ_KORISNIKA_PO_JEZGRI:
        clanci = set()
        node_hops_origin=dict()
        random_node = random.choice(all_nodes_ids) # random node
        node_neighbors = list(maxG.neighbors(random_node))             # svi susjedi

        while len(node_neighbors) == 0:   #izabrani cvor nema susjeda
            random_node = random.choice(all_nodes_ids)
            node_neighbors = list(maxG.neighbors(random_node))

        clanci.add(random_node)
        node_hops_origin[random_node]=0
        num_positive = random.randint(MIN_CLANAKA, MAX_CLANAKA)
        
        clanci, node_hops_origin, tmp_max, avg_pos_hops = get_positives(clanci, num_positive, node_hops_origin)
        hard_neg, avg_neg_hops = get_hard_neg(clanci, node_hops_origin, tmp_max)
        easy_neg= random.choices(all_nodes_ids,k=NUM_EASY_NEG)
        all_articles.append((clanci,hard_neg,easy_neg))
        all_neg_avg_hops.append(avg_neg_hops)
        all_pos_avg_hops.append(avg_pos_hops)
            
            
        if len(all_articles)>=BROJ_KORISNIKA_PO_JEZGRI:
            return all_articles, all_neg_avg_hops, all_pos_avg_hops



            
        

In [45]:
def get_positives(init_clanci, NUM_ARTICLES,node_hops_origin):
    p = 0

    br=0
    all_hops=0
    while p < NUM_ARTICLES :
            random_node = random.choice(list(init_clanci)) # rand node

            last_hops=node_hops_origin[random_node] + 1
            neighbors = set(maxG.neighbors(random_node))  # svi njegovi susjedi
            unknown_neighbors = list(neighbors.difference(init_clanci))  #dohvati ako nisu u člancima postojecim
        
            if len(unknown_neighbors) == 0: #ako su već svi u listi
                continue                 #izaberi drugi random node

            
            
            weights = []
            for n in unknown_neighbors:
                eid = maxG.get_eid(random_node, n)
                weights.append(maxG.es[eid]['weight'])

            max_value = sum(weights)

            for f in range(0, len(weights)):
                weights[f] = weights[f] / max_value
                if f > 0:
                    weights[f] = weights[f] + weights[f - 1]

            weights = list(np.around(np.array(weights), 5))
            pick = random.uniform(0, 1)

            position = np.searchsorted(weights, pick)

            random_node2 = unknown_neighbors[position]
            if random_node2 not in node_hops_origin:
                node_hops_origin[random_node2]=last_hops

            init_clanci.add(random_node2)
            p += 1
            br+=1
            all_hops+=last_hops

            if p >= NUM_ARTICLES:
                tmp_max = np.max(list(node_hops_origin.values())) 
                

           
                        
    
    return init_clanci, node_hops_origin, tmp_max , all_hops/br
    
    

In [47]:
clanci_svih_korisnika = []
aph,anh=[],[]

start_time = datetime.now()
pool = mp.Pool(BROJ_JEZGRI)
for oo in list(range(BROJ_JEZGRI)):
    pool.apply_async(walk, callback=get_articles_from_processes)

pool.close()
pool.join()

end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:01:01.732027


In [48]:
print("positive med and avg: ",np.median(aph),np.average(aph))

print("negative med and avg: ",np.median(anh),np.average(anh))

positive med and avg:  2.0 2.1183202962810594
negative med and avg:  13.75 13.745715009465911


Spremanje

In [122]:
def append_new_user(user_dict,userID,posIDList,h_negIDList,e_negIDList):
    user_dict["users"].append({
                 "userID": userID,
                 "posIDs": list(posIDList),
                 "hard_negIDs": list(h_negIDList),
                 "easy_negIDs": list(e_negIDList)
                 })
    userID +=1
    return user_dict, userID

In [123]:
print("spremanje korisnika u "+USERS_JSON_PATH+"....")

spremanje korisnika u /home/dora/Desktop/recommender/datasets/master_generated/6-8-200000-100_ID@_3/random_walk_6-8-200000-100_ID@_3.json....


In [124]:
clanci_svih_korisnika2 = []
pbar = tq.tqdm(total=len(clanci_svih_korisnika))
for pos, h_neg, e_neg in clanci_svih_korisnika:
    pos_set = set()
    h_neg_set = set()
    e_neg_set = set()
    for p in pos:
        id_art = maxG.vs[p]['name']
        pos_set.add(id_art)
    for n in h_neg:
        id_art = maxG.vs[n]['name']
        h_neg_set.add(id_art)
    for n in e_neg:
        id_art = maxG.vs[n]['name']
        e_neg_set.add(id_art)        
        
        
    clanci_svih_korisnika2.append((pos_set,h_neg_set,e_neg_set))
    pbar.update(1)

pbar.close()

100%|███████████████████████████████| 200016/200016 [00:01<00:00, 122721.87it/s]


In [125]:
userID=0
user_dict = {}
user_dict["users"] = []

for i in range(len(clanci_svih_korisnika2)):
    user_dict,userID = append_new_user(user_dict, userID, clanci_svih_korisnika2[i][0],clanci_svih_korisnika2[i][1],clanci_svih_korisnika2[i][2]) 

In [126]:
with open(USERS_JSON_PATH, 'w') as outfile:
    json.dump(user_dict["users"],outfile,indent=3)

In [127]:
VERSION

'6-8-200000-100_ID@_3'

In [128]:
USERS_JSON_PATH

'/home/dora/Desktop/recommender/datasets/master_generated/6-8-200000-100_ID@_3/random_walk_6-8-200000-100_ID@_3.json'